--- Day 17: Conway Cubes ---
As your flight slowly drifts through the sky, the Elves at the Mythical Information Bureau at the North Pole contact you. They'd like some help debugging a malfunctioning experimental energy source aboard one of their super-secret imaging satellites.

The experimental energy source is based on cutting-edge technology: a set of Conway Cubes contained in a pocket dimension! When you hear it's having problems, you can't help but agree to take a look.

The pocket dimension contains an infinite 3-dimensional grid. At every integer 3-dimensional coordinate (x,y,z), there exists a single cube which is either active or inactive.

In the initial state of the pocket dimension, almost all cubes start inactive. The only exception to this is a small flat region of cubes (your puzzle input); the cubes in this region start in the specified active (#) or inactive (.) state.

The energy source then proceeds to boot up by executing six cycles.

Each cube only ever considers its neighbors: any of the 26 other cubes where any of their coordinates differ by at most 1. For example, given the cube at x=1,y=2,z=3, its neighbors include the cube at x=2,y=2,z=2, the cube at x=0,y=2,z=3, and so on.

During a cycle, all cubes simultaneously change their state according to the following rules:

If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.  
If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.  
The engineers responsible for this experimental energy source would like you to simulate the pocket dimension and determine what the configuration of cubes should be at the end of the six-cycle boot process.

For example, consider the following initial state:  

.#.  
..#  
\###  
Even though the pocket dimension is 3-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1 region of the 3-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z coordinate (and the frame of view follows the active cells in each cycle):

Before any cycles:
  
z=0  
.#.  
..#  
\###  
  
  
After 1 cycle:  
  
z=-1    
#..  
..#  
.#.  
  
z=0  
#.#  
.##  
.#.  
  
z=1  
#..  
..#  
.#.  
  
  
After 2 cycles:  
  
z=-2  
.....  
.....  
..#..  
.....  
.....  
  
z=-1  
..#..  
.#..#  
....#  
.#...  
.....  
  
z=0  
##...  
##...  
#....  
....#  
.###.  

z=1  
..#..  
.#..#  
....#  
.#...  
.....  
  
z=2  
.....  
.....  
..#..  
.....  
.....  
  
  
After 3 cycles:  
  
z=-2  
.......  
.......  
..##...  
..###..  
.......  
.......  
.......  
  
z=-1  
..#....  
...#...  
#......  
.....##  
.#...#.  
..#.#..  
...#...  
  
z=0  
...#...  
.......  
#......  
.......  
.....##  
.##.#..  
...#...  

z=1  
..#....  
...#...  
#......  
.....##  
.#...#.  
..#.#..  
...#...  
  
z=2  
.......  
.......  
..##...  
..###..  
.......  
.......  
.......  
After the full six-cycle boot process completes, 112 cubes are left in the active state.  

Starting with your given initial configuration, simulate six cycles. How many cubes are left in the active state after the sixth cycle?

In [51]:
# starting_pos = """
# .#.
# ..#
# ###
# """

starting_pos = """
.#.##..#
....#.##
##.###..
.#.#.###
#.#.....
.#..###.
.#####.#
#..####.
"""

n_cycle = 6
verbose = False

In [52]:
import numpy as np
import itertools

cell_auto = [[list(line) for line in starting_pos.strip().split('\n')]]
cell_auto = np.where(
    np.array(cell_auto) == '#',
             1,
             0
            )
cell_auto

def get_neighboors(cell_auto) :
    x,y,z = cell_auto.shape
    neighb = np.zeros((x+2,y+2,z+2))
    for i,j,k in itertools.product(range(3), repeat = 3) :
        if not (i == j == k == 1) :
            neighb[i:x+i,j:y+j,k:z+k] += cell_auto
    return neighb

def iter_cell_auto(cell_auto, neighboors):
    
    x,y,z = cell_auto.shape
    next_cell = np.zeros(neighboors.shape)
    next_cell[1:-1, 1:-1, 1:-1] += cell_auto
    next_cell = (neighboors == 3) | ((next_cell == 1) & (neighboors == 2))
    next_cell = np.where(next_cell,1,0)
    return next_cell

def trim_cell (cell, axis, reverse = False) :
    if axis :
        cell = cell.swapaxes(0,axis)
    if reverse :
        cell = cell[::-1,:,:]
    
    while cell[0,:,:].sum() == 0 :
        cell = cell[1:,:,:]
    
    if axis :
        cell = cell.swapaxes(0,axis)
    if reverse :
        return cell[::-1,:,:]
    else : 
        return trim_cell (cell, axis, True)

if verbose : 
    print(f"at cycle n°{0} we have :")
    print(cell_auto)
    print('')

for n_c in range(n_cycle) :
    neighboors = get_neighboors(cell_auto)
    cell_auto = iter_cell_auto(cell_auto, neighboors)
    for axis in range(3) :
        cell_auto = trim_cell(cell_auto,axis)
    if verbose : 
        print(f"at cycle n°{n_c+1} we have :")
        print(cell_auto)
        print('')
print(f"at the end of {n_c+1} cycles, we have {cell_auto.sum()} activated cells")

at the end of 6 cycles, we have 338 activated cells


## Part Two ---
For some reason, your simulated results don't match what the experimental energy source engineers expected. Apparently, the pocket dimension actually has four spatial dimensions, not three.

The pocket dimension contains an infinite 4-dimensional grid. At every integer 4-dimensional coordinate (x,y,z,w), there exists a single cube (really, a hypercube) which is still either active or inactive.

Each cube only ever considers its neighbors: any of the 80 other cubes where any of their coordinates differ by at most 1. For example, given the cube at x=1,y=2,z=3,w=4, its neighbors include the cube at x=2,y=2,z=3,w=3, the cube at x=0,y=2,z=3,w=4, and so on.

The initial state of the pocket dimension still consists of a small flat region of cubes. Furthermore, the same rules for cycle updating still apply: during each cycle, consider the number of active neighbors of each cube.

For example, consider the same initial state as in the example above. Even though the pocket dimension is 4-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1x1 region of the 4-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z and w coordinate:

Before any cycles:  

z=0, w=0  
.#.  
..#  
\###  


After 1 cycle:

z=-1, w=-1  
#..  
..#  
.#.  

z=0, w=-1  
#..  
..#  
.#.  

z=1, w=-1  
#..  
..#  
.#.  

z=-1, w=0  
#..  
..#    
.#.  

z=0, w=0  
#.#  
.##  
.#.  

z=1, w=0  
#..  
..#  
.#.  

z=-1, w=1  
#..  
..#  
.#.  

z=0, w=1  
#..  
..#    
.#.  

z=1, w=1  
#..  
..#  
.#.  


After 2 cycles:  

z=-2, w=-2  
.....  
.....  
..#..  
.....  
.....  

z=-1, w=-2  
.....  
.....  
.....  
.....  
.....  

z=0, w=-2  
###..  
##.##  
#...#  
.#..#  
.###.  
  
z=1, w=-2  
.....  
.....  
.....  
.....  
.....  

z=2, w=-2  
.....  
.....    
..#..  
.....  
.....  

z=-2, w=-1  
.....  
.....  
.....  
.....  
.....  

z=-1, w=-1  
.....  
.....  
.....  
.....  
.....  

z=0, w=-1
.....
.....
.....
.....
.....

z=1, w=-1  
.....  
.....  
.....  
.....  
.....  

z=2, w=-1  
.....  
.....  
.....  
.....  
.....  

z=-2, w=0  
###..  
##.##  
#...#  
.#..#  
.###.  

z=-1, w=0  
.....  
.....  
.....  
.....  
.....  

z=0, w=0  
.....  
.....  
.....  
.....  
.....  

z=1, w=0  
.....  
.....  
.....  
.....  
.....  

z=2, w=0  
###..  
##.##  
#...#  
.#..#  
.###.  

z=-2, w=1  
.....  
.....  
.....  
.....  
.....  

z=-1, w=1  
.....  
.....  
.....  
.....  
.....  

z=0, w=1  
.....  
.....  
.....  
.....  
.....  
 
z=1, w=1  
.....  
.....  
.....  
.....  
.....   

z=2, w=1  
.....  
.....  
.....  
.....  
.....  

z=-2, w=2  
.....   
.....  
..#..  
.....  
.....    

z=-1, w=2  
.....  
.....  
.....  
.....  
.....   

z=0, w=2  
###..  
##.##  
#...#  
.#..#  
.###.   

z=1, w=2   
.....  
.....  
.....  
.....  
.....  
  
z=2, w=2  
.....  
.....  
..#..  
.....  
.....  
After the full six-cycle boot process completes, 848 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles in a 4-dimensional space. How many cubes are left in the active state after the sixth cycle?

In [61]:
# starting_pos = """
# .#.
# ..#
# ###
# """

starting_pos = """
.#.##..#
....#.##
##.###..
.#.#.###
#.#.....
.#..###.
.#####.#
#..####.
"""

n_cycle = 
verbose = False

SyntaxError: invalid syntax (<ipython-input-61-0e75e6fcd04f>, line 18)

In [62]:
import numpy as np
import itertools

cell_auto = [[[list(line) for line in starting_pos.strip().split('\n')]]]
cell_auto = np.where(
    np.array(cell_auto) == '#',
             1,
             0
            )


def get_neighboors(cell_auto) :
    x,y,z,w = cell_auto.shape
    neighb = np.zeros((x+2,y+2,z+2,w+2))
    for i,j,k,l in itertools.product(range(3), repeat = 4) :
        if not (i == j == k == l == 1) :
            neighb[i:x+i,j:y+j,k:z+k,l:w+l] += cell_auto
    return neighb

def iter_cell_auto(cell_auto, neighboors):
    
    next_cell = np.zeros(neighboors.shape)
    next_cell[1:-1, 1:-1, 1:-1, 1:-1] += cell_auto
    next_cell = (neighboors == 3) | ((next_cell == 1) & (neighboors == 2))
    next_cell = np.where(next_cell,1,0)
    return next_cell

def trim_cell (cell, axis, reverse = False) :
    if axis :
        cell = cell.swapaxes(0,axis)
    if reverse :
        cell = cell[::-1,:,:,:]
    
    while cell[0,:,:,:].sum() == 0 :
        cell = cell[1:,:,:,:]
    
    if axis :
        cell = cell.swapaxes(0,axis)
    if reverse :
        return cell[::-1,:,:,:]
    else : 
        return trim_cell (cell, axis, True)

if verbose : 
    print(f"at cycle n°{0} we have :")
    print(cell_auto)
    print('')

for n_c in range(n_cycle) :
    neighboors = get_neighboors(cell_auto)
    cell_auto = iter_cell_auto(cell_auto, neighboors)
    for axis in range(4) :
        cell_auto = trim_cell(cell_auto,axis)
    if verbose : 
        print(f"at cycle n°{n_c+1} we have :")
        print(cell_auto)
        print('')
print(f"at the end of {n_c+1} cycles, we have {cell_auto.sum()} activated cells")

at the end of 6 cycles, we have 2440 activated cells
